In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
#1.下载mnist数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot=True) #让label转换成one-hot格式

#2对数据进行操作
#2.1分成多个batch
batch_size = 100
n_batch = mnist.train.num_examples // batch_size  #//整除

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


# 命名空间
with tf.name_scope(name='input'):

In [4]:
#2.2为data 和 label 创建plcaeholder   #！None 具体多少数量之后指定
#命名空间
with tf.name_scope(name='input'):
    data = tf.placeholder(tf.float32,[None,784],name='x-input')
    label = tf.placeholder(tf.float32,[None,10],name='y-input')
keep_prob = tf.placeholder(tf.float32)
learn = tf.Variable(0.001,dtype=tf.float32)

In [6]:
#3.创建神经网络
#3.1权重参数 和 偏置项

#输出层
with tf.name_scope('layer'):
    with tf.name_scope('weight'):
        W = tf.Variable(tf.truncated_normal([784,10],stddev=0.1),name='W')
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10])+0.1,name='b')
    #3.2softmax 为分类求出概率
    with tf.name_scope('mul_add'):
        mul_add = tf.matmul(data,W)+b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(mul_add) 

#3.3使用二次代价函数计算loss值
# loss = tf.reduce_mean(tf.square(prediction-label))
#使用交叉熵 计算loss\
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=label,logits=prediction))

#3.4使用梯度下降法
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learn).minimize(loss)

#4.求出准确率
#4.1 得出一个判断正确与否的 布尔类型list
#! arg_max（） 返回一维张量中最大的索引值 1表示按行比较   这里 前一个就是取出one-hot的那个1所在的位置 后一个就是取出概率值最大的位置
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(label,1),tf.argmax(prediction,1))
#4.2 计算正确率   将布尔型列表转换成 0 1 求出平均值就是正确率
    with tf.name_scope('accuracy'):
        accuracy = (tf.reduce_mean(tf.cast(correct_prediction,tf.float32)))



# 在session中把图保存下来

In [7]:
#5. session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #将图保存下来
    tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(1):
        sess.run(tf.assign(learn,learn*(0.95**epoch))) 
        for batch in range(n_batch):
            #每次向下取出一个batch的数据
            train_data,train_label = mnist.train.next_batch(batch_size) #？为什么这里mnist 有 train方法...
            sess.run(train_step,feed_dict={data:train_data,label:train_label,keep_prob:1.0})
            #传入测试集的数据和标签
        test_acc = sess.run(accuracy,feed_dict={data:mnist.test.images,label:mnist.test.labels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={data:mnist.train.images,label:mnist.train.labels,keep_prob:1.0})
        print(epoch,"train:",train_acc,"test:",test_acc)

0 train: 0.8868727 test: 0.8925
